# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [2]:
# Drop unnecessary columns
new_df = df.drop(columns=['timestamp'])
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int64  
 1   movieId  100836 non-null  int64  
 2   rating   100836 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB


It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [3]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 
reader = Reader(rating_scale=(new_df['rating'].min(), new_df['rating'].max()))
data = Dataset.load_from_df(new_df[['userId', 'movieId', 'rating']], reader)
print(data)


Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [4]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [19]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [20]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
# Define the parameter grid for SVD


In [21]:
# print out optimal parameters for SVD after GridSearch
print(f"Best RMSE for SVD: {grid_search.best_score['rmse']}")
print(f"Best parameters for SVD: {grid_search.best_params['rmse']}")

AttributeError: 'GridSearchCV' object has no attribute 'best_score'

In [22]:
# cross validating with KNNBasic
knn_basic = KNNBasic()
cv_results_knn_basic = cross_validate(knn_basic, data, measures=['rmse'], cv=5, verbose=True)
print(f"Average RMSE for KNNBasic: {np.mean(cv_results_knn_basic['test_rmse'])}")

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9522  0.9468  0.9408  0.9436  0.9528  0.9472  0.0047  
Fit time          0.17    0.20    0.20    0.20    0.24    0.20    0.02    
Test time         1.25    1.76    1.55    1.45    1.41    1.48    0.17    
Average RMSE for KNNBasic: 0.947244488614474


In [23]:
# print out the average RMSE score for the test set
print(f"Average RMSE for KNNBaseline: {np.mean(cv_results_knn_baseline['test_rmse'])}")

NameError: name 'cv_results_knn_baseline' is not defined

In [24]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline()
cv_results_knn_baseline = cross_validate(knn_baseline, data, measures=['rmse'], cv=5, verbose=True)
cv_results_knn_baseline


Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8740  0.8764  0.8773  0.8766  0.8721  0.8753  0.0020  
Fit time          0.47    0.64    0.58    0.52    0.56    0.55    0.06    
Test time         1.67    1.86    1.92    1.84    1.76    1.81    0.09    


{'test_rmse': array([0.87400713, 0.87644062, 0.87734318, 0.87662822, 0.87211758]),
 'fit_time': (0.4726588726043701,
  0.6389279365539551,
  0.578463077545166,
  0.516047477722168,
  0.5611999034881592),
 'test_time': (1.6735949516296387,
  1.8632385730743408,
  1.9202451705932617,
  1.8390963077545166,
  1.7587780952453613)}

In [10]:
# print out the average score for the test set


Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [25]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [26]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [27]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [28]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.0403455119798073, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [35]:
def movie_rater(movie_df,num, genre=None):
    if genre:
        filtered_movies = movie_df[movie_df['genres'].str.contains(genre, case=False, na=False)]
    else:
        filtered_movies = movie_df
    
    # Initialize an empty list to store the ratings
    rating_list = []
    
    # Loop until the required number of ratings are collected
    for _ in range(num):
        # Randomly select a movie from the filtered list
        movie = filtered_movies.sample(n=1).iloc[0]
        
        # Ask the user to rate the movie
        print(f"How do you rate this movie on a scale of 1-5, press 'n' if you have not seen it:")
        print(f"Movie: {movie['title']} ({movie['genres']})")
        
        # Get user input for rating
        rating = input("Your rating (1-5 or n to skip): ").strip()
        
        # If the user skips, continue to the next movie
        if rating.lower() == 'n':
            continue
        
        # Validate if the input is a number between 1 and 5
        try:
            rating = float(rating)
            if 1 <= rating <= 5:
                rating_list.append({'userId': 1, 'movieId': movie['movieId'], 'rating': rating})
            else:
                print("Please enter a rating between 1 and 5.")
        except ValueError:
            print("Invalid input. Please enter a number between 1 and 5, or 'n' to skip.")
    
    return rating_list

        

In [37]:
# try out the new function here!

      movieId                   title          genres
6579    55245  Good Luck Chuck (2007)  Comedy|Romance
How do you rate this movie on a scale of 1-5, press n if you have not seen :
5
      movieId                       title          genres
1873     2491  Simply Irresistible (1999)  Comedy|Romance
How do you rate this movie on a scale of 1-5, press n if you have not seen :
4
      movieId                  title  genres
3459     4718  American Pie 2 (2001)  Comedy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
4
      movieId             title                   genres
4160     5990  Pinocchio (2002)  Children|Comedy|Fantasy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
3


If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [38]:
user_rating = [
    {'userId': 1, 'movieId': 1, 'rating': 4.5},
    {'userId': 1, 'movieId': 2, 'rating': 3.0},
    {'userId': 2, 'movieId': 1, 'rating': 5.0},
    {'userId': 2, 'movieId': 3, 'rating': 4.0},
    {'userId': 3, 'movieId': 4, 'rating': 3.5}
]
# Convert user ratings to DataFrame
user_ratings_df = pd.DataFrame(user_rating)

# Display the user ratings
print(user_ratings_df)

   userId  movieId  rating
0       1        1     4.5
1       1        2     3.0
2       2        1     5.0
3       2        3     4.0
4       3        4     3.5


### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [39]:
## add the new ratings to the original ratings DataFrame
user_rating = [
    {'userId': 1, 'movieId': 1, 'rating': 4.5},
    {'userId': 1, 'movieId': 2, 'rating': 3.0},
    {'userId': 2, 'movieId': 1, 'rating': 5.0},
    {'userId': 2, 'movieId': 3, 'rating': 4.0},
    {'userId': 3, 'movieId': 4, 'rating': 3.5}
]

user_ratings_df = pd.DataFrame(user_rating)

# Sample movie data
movie_data = {
    'movieId': [1, 2, 3, 4],
    'title': ['Movie A', 'Movie B', 'Movie C', 'Movie D'],
    'genres': ['Comedy|Drama', 'Comedy|Action', 'Action|Adventure', 'Comedy|Romance']
}

movie_df = pd.DataFrame(movie_data)

# Add new user ratings into the original ratings DataFrame
# Assuming original_ratings_df is the original dataset containing all previous ratings
original_ratings_df = pd.DataFrame({
    'userId': [1, 1, 2, 2],
    'movieId': [1, 2, 3, 4],
    'rating': [5, 4, 4, 3]
})

# Merge the original ratings with the new user ratings
combined_ratings_df = pd.concat([original_ratings_df, user_ratings_df], ignore_index=True)
print(combined_ratings_df)

   userId  movieId  rating
0       1        1     5.0
1       1        2     4.0
2       2        3     4.0
3       2        4     3.0
4       1        1     4.5
5       1        2     3.0
6       2        1     5.0
7       2        3     4.0
8       3        4     3.5


In [41]:
# train a model using the new combined DataFrame
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.2)

# Train the model using the SVD algorithm
svd = SVD()
svd.fit(trainset)

In [42]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
# We need to get all movieIds that the user has not rated yet
all_movie_ids = movie_df['movieId'].tolist()

# Get a list of movieIds the user has rated
rated_movie_ids = combined_ratings_df[combined_ratings_df['userId'] == 1]['movieId'].tolist()

# Get the list of movies the user hasn't rated
unrated_movie_ids = [movie_id for movie_id in all_movie_ids if movie_id not in rated_movie_ids]

# Generate predictions for the unrated movies
predictions = [svd.predict(1, movie_id) for movie_id in unrated_movie_ids]

# Create a list of tuples (movieId, predicted_rating)
predicted_ratings = [(pred.iid, pred.est) for pred in predictions]


In [43]:
# order the predictions from highest to lowest rated

ranked_movies = sorted(predicted_ratings, key=lambda x: x[1], reverse=True)

# Get the top N recommendations
top_n = 3
top_n_movies = ranked_movies[:top_n]

# Display the top N recommendations with movie titles
top_n_recommendations = []
for movie_id, _ in top_n_movies:
    movie_title = movie_df[movie_df['movieId'] == movie_id]['title'].values[0]
    top_n_recommendations.append(movie_title)

print(f"Top {top_n} movie recommendations: {top_n_recommendations}")

Top 3 movie recommendations: ['Movie C', 'Movie D']


 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [57]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-57-43ff94c0aecd>, line 3)

## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.